In [ ]:
# !pip install --quiet statsmodels 
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import yfinance as yf

# Get a macro time series — e.g., 10Y Treasury yield
ticker = '^TNX'
ticker = 'SPY'

data = yf.download(ticker, start="2010-01-01", end="2024-01-01")["Close"].dropna()

# Apply ADF test
result = adfuller(data)

# Print results
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Critical Values:", result[4])

if result[1] < 0.05:
    print("✅ Reject null — series is stationary (I(0))")
else:
    print("❌ Fail to reject null — series has a unit root (I(1))")


In [ ]:
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt

# Black-Scholes-Merton Greek Calculations (European Options)
def bsm_greeks(S, K, T, r, q, sigma, option_type='call'):
    # S: spot price
    # K: strike price
    # T: time to maturity (years)
    # r: risk-free rate
    # q: dividend yield
    # sigma: volatility of underlying asset

    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    pdf_d1 = si.norm.pdf(d1, 0.0, 1.0)
    cdf_d1 = si.norm.cdf(d1, 0.0, 1.0)
    cdf_d2 = si.norm.cdf(d2, 0.0, 1.0)
    cdf_minus_d1 = si.norm.cdf(-d1, 0.0, 1.0)
    cdf_minus_d2 = si.norm.cdf(-d2, 0.0, 1.0)

    if option_type == 'call':
        delta = np.exp(-q * T) * cdf_d1
        theta = (-S * pdf_d1 * sigma * np.exp(-q * T)) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * cdf_d2 + q * S * np.exp(-q * T) * cdf_d1
        theta = theta / 365 # Per day
    elif option_type == 'put':
        delta = np.exp(-q * T) * (cdf_d1 - 1)
        theta = (-S * pdf_d1 * sigma * np.exp(-q * T)) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * cdf_minus_d2 - q * S * np.exp(-q * T) * cdf_minus_d1
        theta = theta / 365 # Per day
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    gamma = np.exp(-q * T) * pdf_d1 / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * pdf_d1 * np.sqrt(T)
    vega = vega / 100 # Per 1% change in vol

    return delta, gamma, vega, theta

# Parameters
S0 = 100
T = 0.25
r = 0.02
q = 0.0
sigma = 0.20

# Risk Reversal Parameters
K_call_rr = 105
K_put_rr = 95

# Bull Call Spread Parameters
K_low_bcs = 95
K_high_bcs = 105

# Underlying Price Range for Plotting
S_range = np.linspace(80, 120, 101)

# --- Calculate Risk Reversal Greeks ---
delta_call_rr, gamma_call_rr, vega_call_rr, theta_call_rr = bsm_greeks(S_range, K_call_rr, T, r, q, sigma, 'call')
delta_put_rr, gamma_put_rr, vega_put_rr, theta_put_rr = bsm_greeks(S_range, K_put_rr, T, r, q, sigma, 'put')

# Long RR = Long Call + Short Put
rr_delta = delta_call_rr - delta_put_rr
rr_gamma = gamma_call_rr - gamma_put_rr # Note: Gamma Put is same as Gamma Call at same strike
rr_vega = vega_call_rr - vega_put_rr
rr_theta = theta_call_rr - theta_put_rr

# --- Calculate Bull Call Spread Greeks ---
delta_low_bcs, gamma_low_bcs, vega_low_bcs, theta_low_bcs = bsm_greeks(S_range, K_low_bcs, T, r, q, sigma, 'call')
delta_high_bcs, gamma_high_bcs, vega_high_bcs, theta_high_bcs = bsm_greeks(S_range, K_high_bcs, T, r, q, sigma, 'call')

# BCS = Long Lower Call + Short Higher Call
bcs_delta = delta_low_bcs - delta_high_bcs
bcs_gamma = gamma_low_bcs - gamma_high_bcs
bcs_vega = vega_low_bcs - vega_high_bcs
bcs_theta = theta_low_bcs - theta_high_bcs


# --- Plotting ---
fig, axs = plt.subplots(2, 4, figsize=(20, 10), sharex=True)
fig.suptitle('Greek Profiles Comparison (T=0.25y, Vol=20%, r=2%)', fontsize=16)

# Risk Reversal Plots (Row 1)
axs[0, 0].plot(S_range, rr_delta, label=f'Long {K_call_rr}C / Short {K_put_rr}P')
axs[0, 0].set_title('Risk Reversal: Delta')
axs[0, 0].axvline(K_put_rr, color='r', linestyle='--', lw=1, label=f'Put Strike ({K_put_rr})')
axs[0, 0].axvline(K_call_rr, color='g', linestyle='--', lw=1, label=f'Call Strike ({K_call_rr})')
axs[0, 0].set_ylabel('Delta')
axs[0, 0].grid(True)
axs[0, 0].legend()

axs[0, 1].plot(S_range, rr_gamma)
axs[0, 1].set_title('Risk Reversal: Gamma')
axs[0, 1].axvline(K_put_rr, color='r', linestyle='--', lw=1)
axs[0, 1].axvline(K_call_rr, color='g', linestyle='--', lw=1)
axs[0, 1].set_ylabel('Gamma')
axs[0, 1].grid(True)

axs[0, 2].plot(S_range, rr_vega)
axs[0, 2].set_title('Risk Reversal: Vega')
axs[0, 2].axvline(K_put_rr, color='r', linestyle='--', lw=1)
axs[0, 2].axvline(K_call_rr, color='g', linestyle='--', lw=1)
axs[0, 2].set_ylabel('Vega (per 1% vol change)')
axs[0, 2].grid(True)

axs[0, 3].plot(S_range, rr_theta)
axs[0, 3].set_title('Risk Reversal: Theta')
axs[0, 3].axvline(K_put_rr, color='r', linestyle='--', lw=1)
axs[0, 3].axvline(K_call_rr, color='g', linestyle='--', lw=1)
axs[0, 3].set_ylabel('Theta (per day)')
axs[0, 3].grid(True)

# Bull Call Spread Plots (Row 2)
axs[1, 0].plot(S_range, bcs_delta, label=f'Long {K_low_bcs}C / Short {K_high_bcs}C')
axs[1, 0].set_title('Bull Call Spread: Delta')
axs[1, 0].axvline(K_low_bcs, color='b', linestyle='--', lw=1, label=f'Low Strike ({K_low_bcs})')
axs[1, 0].axvline(K_high_bcs, color='m', linestyle='--', lw=1, label=f'High Strike ({K_high_bcs})')
axs[1, 0].set_xlabel('Underlying Price')
axs[1, 0].set_ylabel('Delta')
axs[1, 0].grid(True)
axs[1, 0].legend()


axs[1, 1].plot(S_range, bcs_gamma)
axs[1, 1].set_title('Bull Call Spread: Gamma')
axs[1, 1].axvline(K_low_bcs, color='b', linestyle='--', lw=1)
axs[1, 1].axvline(K_high_bcs, color='m', linestyle='--', lw=1)
axs[1, 1].set_xlabel('Underlying Price')
axs[1, 1].set_ylabel('Gamma')
axs[1, 1].grid(True)

axs[1, 2].plot(S_range, bcs_vega)
axs[1, 2].set_title('Bull Call Spread: Vega')
axs[1, 2].axvline(K_low_bcs, color='b', linestyle='--', lw=1)
axs[1, 2].axvline(K_high_bcs, color='m', linestyle='--', lw=1)
axs[1, 2].set_xlabel('Underlying Price')
axs[1, 2].set_ylabel('Vega (per 1% vol change)')
axs[1, 2].grid(True)

axs[1, 3].plot(S_range, bcs_theta)
axs[1, 3].set_title('Bull Call Spread: Theta')
axs[1, 3].axvline(K_low_bcs, color='b', linestyle='--', lw=1)
axs[1, 3].axvline(K_high_bcs, color='m', linestyle='--', lw=1)
axs[1, 3].set_xlabel('Underlying Price')
axs[1, 3].set_ylabel('Theta (per day)')
axs[1, 3].grid(True)


plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
# Instead of plt.show(), we will describe the plots as AI cannot display them directly.
# plt.show() 
# print("Plot generation complete. AI will now describe the plots.")